# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [85]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [86]:
cities_data = os.path.join('..', 'output_data', 'cities.csv')
cities_df = pd.read_csv(cities_data)
cities_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [87]:
gmaps.configure(api_key = g_key)

In [116]:
weight = cities_df['Humidity'].astype(float)
locations = cities_df[['Lat', 'Lng']].astype(float)
cities_df = cities_df.dropna()
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'    
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=weight,
                             dissipating=False, max_intensity=100,
                             point_radius=1)
fig.add_layer(heat_layer)
#Will not print out my map!
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [56]:
nice_weather = cities_df.loc[(cities_df['Max Temp'] < 80) &
                            (cities_df['Max Temp'] > 70)]
nice_weather = cities_df.loc[cities_df['Wind Speed'] <10]
nice_weather = cities_df.loc[cities_df['Cloudiness'] == 0]
nice_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
11,11,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17
13,13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26
26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
...,...,...,...,...,...,...,...,...,...,...
527,527,tomatlan,0,MX,1558378894,59,19.93,-105.25,84.66,7.90
533,533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [67]:
hotel_df = nice_weather

In [72]:
hotel_df['Hotel Name'] = ''

C:\Users\gatli\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [141]:
hotels = []
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    #I have no idea why it will not add the results to my list. When I print "results" it shows the list that I am looking for, I just can not index the names!
    hotels.append(results[0]['name'])
hotel_df['Hotel Name'] = hotels
hotel_df




        


IndexError: list index out of range

In [143]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cities_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [144]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…